## Pareto Q-Learning with Deep Learning (or **Deep Pareto Q-Learning**)

This notebook contains an implementation of an **offline** & **continuous** version of the [Pareto Q-Learning](https://jmlr.org/papers/volume15/vanmoffaert14a/vanmoffaert14a.pdf) (PQL) algorithm, a multi-objective reinforcement learning algorithm. The implementation is inspired from the [MORL baselines PQL](https://github.com/LucasAlegre/morl-baselines/blob/main/morl_baselines/multi_policy/pareto_q_learning/pql.py) implementation.

### Pareto Q-Learning (PQL) with Deep Learning: Big Picture 🖼️

Imagine you're teaching a robot to make decisions where there isn't just one "best" outcome, but several good outcomes that represent different trade-offs (e.g., getting a task done quickly vs. using minimal energy). PQL helps the robot learn about these trade-offs.

**Core Idea:** Instead of learning a single Q-value (a score for taking an action in a state), PQL learns a set of Q-value vectors for each state-action pair. Each vector in this set represents a possible long-term trade-off. For instance, if there are two objectives (like speed and energy), a Q-vector might be `[high_speed_score, low_energy_score]`. The goal is to find actions leading to "non-dominated" sets of these Q-vectors - meaning no other action offers a universally better set of trade-offs.

**Adapting to Deep Learning (for continuous states):** Since we can't use tables for continuous states, we use a neural network as a function approximator. This network will predict Q-value vectors.

Following is a code skeleton of the PQL algorithm using deep learning:

```python
import torch
import torch.nn as nn
import numpy as np
import copy

# helper function
def get_non_dominated():
    pass

# helper function
def hypervolume():
    pass


# Neural Network class
class MyNetwork(nn.Module):
    def __init__(...):
        ...

    def forward(...):
        ...


class DeepPQLAgent:
    def __init__(self, network, num_actions, num_objectives, hyperparameters...):
        # Initialize network, target network, optimizer, etc.
        # Store hyperparameters (gamma, learning rate, target update strategy, etc.)
        pass

    def _get_q_vectors_from_network(self, network_to_use, state_tensor):
        # Use the provided network (main or target) to predict Q-vectors for all actions given a state.
        pass

    def _calculate_target(self, rewards_batch, next_states_batch, dones_batch):
        # This is the heart of PQL logic:
        # 1. Use the target network to predict Q-vectors for all actions in `next_states_batch`.
        # 2. For each next state in the batch, determine a set of "good" future Q-vectors
        #    (eg: the non-dominated set among predicted Q-vectors for that next state).
        # 3. From this set, select or construct the specific target Q-vector(s) for the Bellman update.
        #    (eg: based on a strategy like "max per objective," "scalarized," "closest to current prediction").
        # 4. Compute final target: R + gamma * (selected future Q-vector)
        pass

    def train_batch(self, states_batch, actions_batch, rewards_batch, next_states_batch, dones_batch):
        # 1. Get current Q-vector predictions from the main network for (states_batch, actions_batch).
        # 2. Calculate target Q-vectors using `_calculate_target`.
        # 3. Compute loss (eg: MSE) between predicted and target Q-vectors.
        # 4. Backpropagate and update the main network.
        # 5. Periodically update the target network.
        pass

    def select_action(self, state, evaluation_strategy):
        # 1. Use the main network to predict Q-vectors for all actions in the given state.
        # 2. Based on the `evaluation_strategy` (eg: "hypervolume," "scalarized_greedy"),
        #    score each action.
        # 3. Return the action with the best score.
        pass


# 4. Training Loop
#    - Load your offline dataset (states, actions, rewards, next_states, dones).
#    - Instantiate your network and the DeepPQLAgent.
#    - For a number of epochs:
#        - Sample batches from the dataset.
#        - Call agent.train_batch(...)
#        - Optionally, evaluate the agent's performance on a validation set.
```

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import copy
from typing import Callable, List, Optional, Set, Tuple, Dict, Union

print("imports done")

imports done


In [ ]:
# helper function
def get_non_dominated(candidates):
    pass

In [7]:
# helper function
def hypervolume(ref_point, points):
    pass

In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, state_dim: int, num_actions: int, num_objectives: int, hidden_size: int = 128):
        super(SimpleMLP, self).__init__()
        self.num_actions = num_actions
        self.num_objectives = num_objectives
        self.network = nn.Sequential(
            nn.Linear(state_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_actions * num_objectives)
        )

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        flat_q_values = self.network(state)
        # Reshape to (batch_size, num_actions, num_objectives)
        return flat_q_values.view(-1, self.num_actions, self.num_objectives)
    

In [ ]:
class DeepPQLAgent:
    def __init__(self,
                 network: nn.Module,
                 num_actions: int,
                 num_objectives: int,
                 state_dim: int, # Added for clarity, though network might already know
                 ref_point: np.ndarray, # For evaluation scoring
                 gamma: float = 0.9,
                 learning_rate: float = 1e-4,
                 optimizer_type: str = "adam",
                 target_network_update_rule: str = "hard_copy", # "hard_copy" or "polyak"
                 target_network_update_freq: int = 100, # For "hard_copy"
                 polyak_tau: float = 0.005, # For "polyak"
                 target_q_calculation_strategy: str = "dqn_max_per_objective", # Default basic strategy
                 loss_function_type: str = "mse",
                 device: Optional[str] = None,
                 seed: Optional[int] = None):

        if seed is not None:
            torch.manual_seed(seed)
            np.random.seed(seed)

        self.device = torch.device(device)
        # self.device = torch.device(device if device else ("cuda" if torch.cuda.is_available() else "cpu"))
        print(f"DeepPQLAgent using device: {self.device}")

        self.network = network.to(self.device)
        self.target_network = copy.deepcopy(network).to(self.device) # Create a target network
        self.target_network.load_state_dict(self.network.state_dict()) # Initialize with same weights
        self.target_network.eval() # Target network is only for inference, not direct training

        # Store parameters
        self.num_actions = num_actions
        self.num_objectives = num_objectives
        self.state_dim = state_dim
        self.ref_point = np.array(ref_point, dtype=np.float32)
        self.gamma = gamma
        self.lr = learning_rate # Renamed to avoid conflict if passed to optimizer directly
        self.target_update_rule = target_network_update_rule
        self.target_update_freq = target_network_update_freq
        self.polyak_tau = polyak_tau
        self.target_q_strat = target_q_calculation_strategy
        self.loss_type = loss_function_type

        if optimizer_type.lower() == "adam":
            self.optimizer = optim.Adam(self.network.parameters(), lr=self.lr)
        elif optimizer_type.lower() == "rmsprop":
            self.optimizer = optim.RMSprop(self.network.parameters(), lr=self.lr)
        else:
            raise ValueError(f"Unsupported optimizer type: {optimizer_type}")

        if self.loss_type.lower() == "mse":
            self.loss_fn = nn.MSELoss()
        elif self.loss_type.lower() == "smooth_l1":
            self.loss_fn = nn.SmoothL1Loss()
        else:
            raise ValueError(f"Unsupported loss type: {self.loss_type}")

        self.global_step_count = 0
        print("DeepPQLAgent initialized.")


    def _get_q_vectors_from_network(self, network_to_use: nn.Module, state_tensor: torch.Tensor) -> torch.Tensor:
        """
        Uses the provided network to predict Q-vectors for all actions given a state tensor.
        Assumes the network's forward pass is: state_tensor -> all_action_q_vectors.
        Input state_tensor: (batch_size, state_dim)
        Output: (batch_size, num_actions, num_objectives)
        """
        # Network is expected to handle the state_tensor directly and output appropriately.
        # If network_input_type parameter was used (like in PQL_Offline_Deep), that logic would be here.
        # For simplicity here, we assume the network itself is designed for state_only input.
        with torch.inference_mode(): # Important when using target_network or just for inference
            q_vectors_all_actions = network_to_use(state_tensor)
        return q_vectors_all_actions
    

    def _calculate_target(self,
                          rewards_batch: torch.Tensor,      # (batch_size, num_objectives)
                          next_states_batch: torch.Tensor,  # (batch_size, state_dim)
                          dones_batch: torch.Tensor,        # (batch_size, 1)
                          # Optional: current_q_s_a_predictions for strategies like "set_closest_from_nd"
                          current_q_s_a_predictions: Optional[torch.Tensor] = None
                         ) -> torch.Tensor:
        batch_size = rewards_batch.shape[0]
        # Initialize target future Q-vectors with zeros (for terminal states)
        target_future_q_vectors = torch.zeros_like(rewards_batch, device=self.device) # (batch_size, num_objectives)

        # Identify non-terminal next states
        non_terminal_mask = (dones_batch.squeeze(1) == 0) # In PQL_Offline_Deep, dones were float (0.0 for non-terminal)
                                                        # If bool, use `~dones_batch.squeeze(1)`
        
        if torch.any(non_terminal_mask):
            non_terminal_next_states = next_states_batch[non_terminal_mask]

            # Get Q-vectors for all actions in non-terminal next_states using the TARGET NETWORK
            # Output shape: (num_non_terminal, num_actions, num_objectives)
            next_q_all_actions_target_nt = self._get_q_vectors_from_network(self.target_network, non_terminal_next_states)

            if self.target_q_strat == "dqn_max_per_objective":
                # For each objective k, find max_{a'} Q_k(s',a')
                # This results in one target vector per non-terminal next state.
                max_q_values_per_objective_nt, _ = torch.max(next_q_all_actions_target_nt, dim=1) # (num_non_terminal, num_objectives)
                target_future_q_vectors[non_terminal_mask] = max_q_values_per_objective_nt
            
            # --- Other strategies would be implemented here ---
            # Example for a conceptual "scalarized_max" (if preference_sampling_method was a class member)
            # elif self.target_q_strat == "scalarized_max":
            #     if self.preference_sampling_method is None:
            #         raise ValueError("Preference sampling method needed for scalarized_max")
            #     temp_targets = torch.zeros_like(target_future_q_vectors[non_terminal_mask])
            #     for i in range(non_terminal_next_states.shape[0]): # Loop over non-terminal samples
            #         w_np = self.preference_sampling_method() # Get a weight vector
            #         w = torch.tensor(w_np, dtype=torch.float32, device=self.device)
            #         scalarized_q_next_for_sample = torch.sum(next_q_all_actions_target_nt[i] * w, dim=1) # (num_actions)
            #         best_action_idx = torch.argmax(scalarized_q_next_for_sample)
            #         temp_targets[i] = next_q_all_actions_target_nt[i, best_action_idx, :]
            #     target_future_q_vectors[non_terminal_mask] = temp_targets

            # Example for "set_closest_from_nd" (conceptual, requires current_q_s_a_predictions_nt)
            # elif self.target_q_strat == "set_closest_from_nd":
            #     if current_q_s_a_predictions is None:
            #         raise ValueError("`current_q_s_a_predictions` needed for 'set_closest_from_nd'")
            #     current_q_s_a_preds_nt = current_q_s_a_predictions[non_terminal_mask]
            #     temp_targets = torch.zeros_like(target_future_q_vectors[non_terminal_mask])
            #     rewards_nt = rewards_batch[non_terminal_mask] # Need rewards for this specific calculation
            #
            #     for i in range(non_terminal_next_states.shape[0]):
            #         q_vectors_s_prime_np = next_q_all_actions_target_nt[i].cpu().numpy() # (num_actions, num_objectives)
            #         nd_set_future_q_tuples = get_non_dominated({tuple(qv) for qv in q_vectors_s_prime_np})
            #
            #         if not nd_set_future_q_tuples: # Fallback
            #             temp_targets[i], _ = torch.max(next_q_all_actions_target_nt[i], dim=0) # Use dqn_max as fallback
            #             continue
            #
            #         potential_full_targets = [
            #             rewards_nt[i] + self.gamma * torch.tensor(v, dtype=torch.float32, device=self.device)
            #             for v in nd_set_future_q_tuples
            #         ]
            #         current_q_pred_for_sample = current_q_s_a_preds_nt[i]
            #
            #         min_dist = float('inf')
            #         chosen_full_target = potential_full_targets[0] # Default
            #         for pt in potential_full_targets:
            #             dist = torch.norm(current_q_pred_for_sample - pt, p=2)
            #             if dist < min_dist:
            #                 min_dist = dist
            #                 chosen_full_target = pt
            #         # We need to store the future part for the Bellman equation: (chosen_full_target - R) / gamma
            #         temp_targets[i] = (chosen_full_target - rewards_nt[i]) / self.gamma
            #
            #     target_future_q_vectors[non_terminal_mask] = temp_targets

            else:
                # Fallback or error for unimplemented strategies
                print(f"Warning: Target Q strategy '{self.target_q_strat}' not fully implemented or recognized. Using dqn_max_per_objective as fallback.")
                max_q_values_per_objective_nt, _ = torch.max(next_q_all_actions_target_nt, dim=1)
                target_future_q_vectors[non_terminal_mask] = max_q_values_per_objective_nt
                
        # Bellman equation: Target = R + gamma * Target_Future_Q (where future Q is 0 for terminal states)
        # dones_batch is (batch_size, 1), 0 for non-terminal, 1 for terminal.
        # So, (1.0 - dones_batch.squeeze(1)) is 1 for non-terminal, 0 for terminal.
        # We need to unsqueeze it to (batch_size, 1) to multiply with (batch_size, num_objectives)
        is_non_terminal_multiplier = (1.0 - dones_batch.squeeze(1)).unsqueeze(1)
        final_target_q_batch = rewards_batch + (self.gamma * target_future_q_vectors * is_non_terminal_multiplier)
        
        return final_target_q_batch
    

    def train_batch(self,
                    states_batch: torch.Tensor,         # (batch_size, state_dim)
                    actions_batch: torch.Tensor,        # (batch_size, 1) holding action indices
                    rewards_batch: torch.Tensor,        # (batch_size, num_objectives)
                    next_states_batch: torch.Tensor,    # (batch_size, state_dim)
                    dones_batch: torch.Tensor           # (batch_size, 1) bool or float (0 for non-terminal, 1 for terminal)
                   ):
        self.network.train() # Set network to training mode

        # 1. Get current Q-vector predictions from the MAIN NETWORK
        #    Output of _get_q_vectors_from_network: (batch_size, num_actions, num_objectives)
        predicted_q_all_actions = self._get_q_vectors_from_network(self.network, states_batch)

        # Select the Q-vector for the specific action taken in the batch
        # actions_batch needs to be shaped correctly for gather: (batch_size, 1, 1) and expanded
        # actions_batch (original): (batch_size, 1)
        # Unsqueeze for gather: (batch_size, 1, 1)
        # Expand to match num_objectives: (batch_size, 1, num_objectives)
        action_indices_expanded = actions_batch.unsqueeze(-1).expand(-1, -1, self.num_objectives)
        
        # Q(s,a) for the actions taken
        current_q_s_a_predictions = predicted_q_all_actions.gather(dim=1, index=action_indices_expanded).squeeze(1) # (batch_size, num_objectives)

        # 2. Calculate target Q-vectors using `_calculate_target`
        #    The target values should not have gradients flowing back through them.
        with torch.inference_mode():
            # Pass current_q_s_a_predictions if needed by the target strategy
            target_q_values = self._calculate_target(rewards_batch, next_states_batch, dones_batch,
                                                     current_q_s_a_predictions if self.target_q_strat == "set_closest_from_nd" else None)

        # 3. Compute loss
        loss = self.loss_fn(current_q_s_a_predictions, target_q_values) # target_q_values is already detached due to torch.no_grad()

        # 4. Backpropagate and update the MAIN NETWORK
        self.optimizer.zero_grad()
        loss.backward()
        # Optional: Gradient Clipping
        # torch.nn.utils.clip_grad_norm_(self.network.parameters(), max_norm=1.0)
        self.optimizer.step()

        self.global_step_count += 1

        # 5. Periodically update the TARGET NETWORK
        if self.target_update_rule == "hard_copy" and self.global_step_count % self.target_update_freq == 0:
            self.target_network.load_state_dict(self.network.state_dict())
            # print(f"Step {self.global_step_count}: Target network updated (hard copy).")
        elif self.target_update_rule == "polyak":
            for param, target_param in zip(self.network.parameters(), self.target_network.parameters()):
                target_param.data.copy_(self.polyak_tau * param.data + (1.0 - self.polyak_tau) * target_param.data)
            # No need to print polyak updates every time, they are frequent.

        return loss.item()
    
    
    def select_action(self, state_np: np.ndarray, evaluation_strategy: str = "hypervolume", eval_preference_vector: Optional[np.ndarray] = None) -> Tuple[int, np.ndarray]:
        """
        Selects an action for a given state during evaluation.
        Input state_np: A single state as a NumPy array (state_dim,)
        Returns: (selected_action_index, q_vector_for_selected_action)
        """
        self.network.eval() # Set network to evaluation mode for consistent outputs
        state_tensor = torch.tensor(state_np, dtype=torch.float32, device=self.device).unsqueeze(0) # Add batch dim: (1, state_dim)

        with torch.inference_mode():
            # Q-vectors for all actions: (1, num_actions, num_objectives)
            q_vectors_all_actions_batch = self._get_q_vectors_from_network(self.network, state_tensor)
            # Remove batch dim: (num_actions, num_objectives)
            q_vectors_all_actions = q_vectors_all_actions_batch.squeeze(0)

        # Convert to list of Python tuples of floats for helper functions
        q_value_tuples_list = [tuple(qv.cpu().numpy()) for qv in q_vectors_all_actions]
        
        best_action_idx = 0 # Default action
        
        if evaluation_strategy == "hypervolume":
            if not q_value_tuples_list: # Should not happen if num_actions > 0
                return np.random.randint(self.num_actions), np.zeros(self.num_objectives)
            
            action_scores = np.array([hypervolume(self.ref_point, [vec]) if vec else -float('inf') for vec in q_value_tuples_list])
            
            if np.all(np.isinf(action_scores)) or np.all(np.isnan(action_scores)) or len(action_scores)==0:
                best_action_idx = np.random.randint(self.num_actions) # Fallback
            else:
                best_action_idx = np.nanargmax(action_scores) # nanargmax handles NaNs by ignoring them

        elif evaluation_strategy == "random_from_nd": # Get non-dominated set and pick randomly
            if not q_value_tuples_list:
                return np.random.randint(self.num_actions), np.zeros(self.num_objectives)
            nd_set_tuples = get_non_dominated(set(q_value_tuples_list))
            if not nd_set_tuples:
                 best_action_idx = np.random.randint(self.num_actions) # Fallback
            else:
                nd_action_indices = [i for i, q_tuple in enumerate(q_value_tuples_list) if q_tuple in nd_set_tuples]
                best_action_idx = np.random.choice(nd_action_indices) if nd_action_indices else np.random.randint(self.num_actions)

        elif evaluation_strategy == "scalarized_greedy":
            if eval_preference_vector is None:
                raise ValueError("`eval_preference_vector` must be provided for 'scalarized_greedy' strategy.")
            w_eval = torch.tensor(eval_preference_vector, dtype=torch.float32, device=self.device)
            # q_vectors_all_actions is (num_actions, num_objectives)
            scalarized_q_values = torch.sum(q_vectors_all_actions * w_eval, dim=1) # (num_actions)
            best_action_idx = torch.argmax(scalarized_q_values).item()
        
        # elif evaluation_strategy == "custom":
            # best_action_idx = self._select_action_custom_eval_logic(...)
        else:
            raise ValueError(f"Unknown evaluation_strategy: {evaluation_strategy}")
            
        return best_action_idx, q_vectors_all_actions[best_action_idx].cpu().numpy()
    

    

In [ ]:
class DeepPQLAgent:
    def __init__(self, num_states, num_actions, num_objectives, gamma=0.9):
        self.num_states = num_states
        self.num_actions = num_actions
        self.num_objectives = num_objectives
        self.gamma = gamma      # Discount factor

        # --- Core PQL Data Structures ---
        # 1. Average immediate rewards: R_avg(s,a)
        #    Stores a vector of rewards for each (s,a)
        self.avg_immediate_rewards = [[np.zeros(num_objectives) for _ in range(num_actions)] for _ in range(num_states)]

        # 2. Future value sets: ND_future(s,a)
        #    Stores a set of non-dominated vectors from the next state,
        #    after taking action 'a' in state 's'.
        #    Each vector represents a possible future trade-off.
        #    Initialize with a set containing one zero vector.
        self.future_value_sets = [[{tuple(np.zeros(num_objectives))} for _ in range(num_actions)] for _ in range(num_states)]

        # 3. Visit counts: N(s,a)
        #    Counts how many times we've tried action 'a' in state 's'.
        self.visit_counts = [[0 for _ in range(num_actions)] for _ in range(num_states)]

    def get_q_set(self, state, action):
        """
        Combines immediate reward with discounted future values
        Returns a SET of Q-vectors for (state, action)
        Q_set(s,a) = R_avg(s,a) + gamma * ND_future(s,a)
        """
        pass

    def get_best_future_values_from_state(self, state):
        """
        Calculates V_ND(state) = ND ( Union_{all_actions_in_state} Q_set(state, action) )
        Returns a SET of non-dominated Q-vectors achievable from 'state'
        """
        pass

    def update_from_experience(self, s, a, r_vector, s_prime, terminated):
        """
        The main learning step. Updates the core data structures.
        """
        pass

    def select_action(self, state, epsilon=0.1):
        """
        Chooses an action from 'state' using epsilon-greedy strategy
        Needs a way to score the Q-sets for greedy selection.
        """
        pass

    def track_policy(self, target_q_vector, initial_state, env):
        """
        Given a desired Q-vector (a specific trade-off),
        tries to follow the policy that achieves it.
        """
        pass



In [1]:
def fn(all_vectors):
    def dominates(vec1, vec2):
        """Check if vec1 Pareto dominates vec2"""
        # vec1 dominates vec2 if:
        # 1. vec1 is at least as good in all objectives
        # 2. vec1 is strictly better in at least one objective
        
        at_least_as_good = all(v1 >= v2 for v1, v2 in zip(vec1, vec2))
        strictly_better = any(v1 > v2 for v1, v2 in zip(vec1, vec2))
        
        return at_least_as_good and strictly_better
    
    non_dominated = set()
    for vec1 in all_vectors:
        is_dominated = False
        for vec2 in all_vectors:
            if vec1 != vec2 and dominates(vec2, vec1):
                is_dominated = True
                break
        if not is_dominated:
            non_dominated.add(vec1)
    return non_dominated


q_vecs = [
    (10, -5, -3),
    (8, -5, -3),
    (8, -2, -3),
    (6, -3, -1),
]

print(fn(q_vecs))  # Should print the non-dominated set from the provided vectors

{(6, -3, -1), (8, -2, -3), (10, -5, -3)}
